In [1]:
import pickle
import re

from Levenshtein import ratio
import pandas as pd

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

from tqdm.notebook import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
examples = {}

for i in range(0, 5):
    with open("../../out/definitions.pickle", "rb") as definitions_file:
        with open(f"../../out/llama/examples_{i}_shot_v4.pickle", "rb") as examples_file:
            examples[i] = {}
            cur_definitions = pickle.load(definitions_file)
            cur_examples = pickle.load(examples_file)
            for (d, de), ex in zip(cur_definitions.items(), cur_examples):
                if i == 0:
                    examples[i][d] = (de, [], ex)
                else:
                    examples[i][d] = (de, *ex[:2])
    examples[i] = pd.DataFrame.from_dict(examples[i], orient="index", columns=["definition", "examples", "generated"])
    
examples[0]

,definition,examples,generated
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[],"[""Die meisten von ihnen haben zuvor noch nie e..."
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...",[],"[""Die Katsup-Bottel sind gleich stark wie die ..."
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...",[],"[""Die meiste Zeit verbringen die Schüler im Kl..."
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...",[],"[""Die Sache ist die beste, die ich je gemacht ..."
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...",[],"[""Das Auto fährt schnell wie ein Flugzeug."", ""..."
...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...",[],"[""Ein Mann, der im Vergleich zu seinem Bruder ..."
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...",[],"[""XoderY""]"
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...",[],"[""Wo es regnet, ist es auch dunkel."", ""Wo es l..."
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...",[],"[""X"", ""so"", ""so"", ""groß"", ""als"", ""Y""]"


## Llama-2:
### Anzahl Beispielsätze:

In [3]:
def evaluate_string(s):
    try:
        evaluated_value = eval(s)
        
        if isinstance(evaluated_value, list):
            
            return [str(e) for e in evaluated_value]  # muss String sein!
        
    except (SyntaxError, ValueError):
        return [s.replace("[", "").replace("]", "")]

In [4]:
for example_group in examples.values():
    example_group['evaluated_generated'] = example_group['generated'].apply(evaluate_string)
    example_group['num_generated'] = example_group['evaluated_generated'].apply(len)

examples[0]

,definition,examples,generated,evaluated_generated,num_generated
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[],"[""Die meisten von ihnen haben zuvor noch nie e...",[Die meisten von ihnen haben zuvor noch nie ei...,2
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...",[],"[""Die Katsup-Bottel sind gleich stark wie die ...",[Die Katsup-Bottel sind gleich stark wie die K...,3
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...",[],"[""Die meiste Zeit verbringen die Schüler im Kl...",[Die meiste Zeit verbringen die Schüler im Kla...,5
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...",[],"[""Die Sache ist die beste, die ich je gemacht ...","[Die Sache ist die beste, die ich je gemacht h...",1
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...",[],"[""Das Auto fährt schnell wie ein Flugzeug."", ""...","[Das Auto fährt schnell wie ein Flugzeug., Das...",6
...,...,...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...",[],"[""Ein Mann, der im Vergleich zu seinem Bruder ...","[Ein Mann, der im Vergleich zu seinem Bruder s...",4
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...",[],"[""XoderY""]",[XoderY],1
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...",[],"[""Wo es regnet, ist es auch dunkel."", ""Wo es l...","[Wo es regnet, ist es auch dunkel., Wo es leic...",5
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...",[],"[""X"", ""so"", ""so"", ""groß"", ""als"", ""Y""]","[X, so, so, groß, als, Y]",6


In [5]:
examples[0]["num_generated"].mean()

3.4739336492890995

In [6]:
examples[0]["num_generated"].mode()  # Häufigste Anzahl Beispielsätze (Modalwert)

0    3
Name: num_generated, dtype: int64

### Anzahl unterschiedlicher Beispielsätze

In [7]:
for example_group in examples.values():
    example_group['unique_generated'] = example_group['evaluated_generated'].apply(lambda x: len(set(x)))

In [8]:
examples[0]

,definition,examples,generated,evaluated_generated,num_generated,unique_generated
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[],"[""Die meisten von ihnen haben zuvor noch nie e...",[Die meisten von ihnen haben zuvor noch nie ei...,2,2
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...",[],"[""Die Katsup-Bottel sind gleich stark wie die ...",[Die Katsup-Bottel sind gleich stark wie die K...,3,3
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...",[],"[""Die meiste Zeit verbringen die Schüler im Kl...",[Die meiste Zeit verbringen die Schüler im Kla...,5,5
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...",[],"[""Die Sache ist die beste, die ich je gemacht ...","[Die Sache ist die beste, die ich je gemacht h...",1,1
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...",[],"[""Das Auto fährt schnell wie ein Flugzeug."", ""...","[Das Auto fährt schnell wie ein Flugzeug., Das...",6,6
...,...,...,...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...",[],"[""Ein Mann, der im Vergleich zu seinem Bruder ...","[Ein Mann, der im Vergleich zu seinem Bruder s...",4,4
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...",[],"[""XoderY""]",[XoderY],1,1
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...",[],"[""Wo es regnet, ist es auch dunkel."", ""Wo es l...","[Wo es regnet, ist es auch dunkel., Wo es leic...",5,5
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...",[],"[""X"", ""so"", ""so"", ""groß"", ""als"", ""Y""]","[X, so, so, groß, als, Y]",6,5


In diesen Ausgaben unterscheiden sich die Anzahl der generierten Beispiele und die Anzahl der tatsächlich unterschiedlichen Beispiele:

In [9]:
differing_lines = examples[0][examples[0]['num_generated'] != examples[0]['unique_generated']]
print(len(differing_lines)/len(examples[0]) * 100, "%")
differing_lines

9.47867298578199 %


,definition,examples,generated,evaluated_generated,num_generated,unique_generated
1043,"Die Konstruktion ""Spezifizierung_Entität:Satzw...",[],"[""Der junge Mann mit dem roten Mantel"", ""Der M...","[Der junge Mann mit dem roten Mantel, Der Mann...",4,3
1134,"Die ""Additiv_Koordination:sowohlXalsauchY"" umf...",[],"[""Das Auto fährt sowohl autotox als hybrid."", ...","[Das Auto fährt sowohl autotox als hybrid., Da...",6,1
130,"Die Konstruktion ""Äquativ:Ein_X1_ohne_Y1_ist_w...",[],"[""Das Leben ohne Konsum ist wie ein Leben ohne...",[Das Leben ohne Konsum ist wie ein Leben ohne ...,14,10
1320,"Bei der ""Diminutiv:N-lein-Konstruktion"" handel...",[],"[""Die Katze fährt auf der Füße"", ""Das Auto fäh...","[Die Katze fährt auf der Füße, Das Auto fährt ...",4,2
1461,"Die ""Kategorisierung:Das_Leben_ist_keinNP-Kons...",[],"[""Das Leben ist kein Nicht-Identitäts-Konstruk...",[Das Leben ist kein Nicht-Identitäts-Konstrukt...,6,1
1594,"Die Konstruktion ""Prototyp_Kompositum:Muster-N...",[],"[""Ein prototypisches Schreibtischmodell"", ""Ein...","[Ein prototypisches Schreibtischmodell, Ein pr...",2,1
1624,"Die Konstruktion ""Koordination_morphologisch:K...",[],"[""Das gemusterte Bettmuster"", ""Das gemusterte ...","[Das gemusterte Bettmuster, Das gemusterte Bet...",3,1
1637,"Bei der Konstruktion ""Einordnung_Gattung:Klamm...",[],"[""Die Katze ist ein Säugetier."", ""Die Katze is...","[Die Katze ist ein Säugetier., Die Katze ist e...",2,1
1639,Mit der Konstruktion Einordnung_Nation:Klammer...,[],"[""Wir treffen uns jeden Samstag im Restaurant""...","[Wir treffen uns jeden Samstag im Restaurant, ...",6,4
1643,"Mit der Konstruktion ""Entsprechung_Zeitangabe:...",[],"[""Die Uhrzeit, die ich an einem bestimmten Dat...","[Die Uhrzeit, die ich an einem bestimmten Datu...",4,3


### Tatsächlich neue Inhalte:

In [10]:
stopwords = {s.lower() for s in stopwords.words("german")}  # Stoppwörter werden entfernt, weil die nichts über Wiederholungen aussagen

def calculate_overlap(row):
    generated_set = {w for w in " ".join(row['evaluated_generated']).split() if w.lower() not in stopwords}
    
    if len(generated_set) > 0:
        definition_overlap = len(generated_set.intersection({w for w in row['definition'].split() if w.lower() not in stopwords})) / len(generated_set)
        examples_overlap = len(generated_set.intersection({w for w in " ".join(row['examples']).split() if w.lower() not in stopwords})) / len(generated_set)
        overlap = len(generated_set.intersection({w for w in row['definition'].split() if w.lower() not in stopwords} | {w for w in " ".join(row['examples']).split() if w.lower() not in stopwords})) / len(generated_set)
    else:
        definition_overlap = 0.0
        examples_overlap = 0.0
        overlap = 0.0
    
    return definition_overlap, examples_overlap, overlap

In [11]:
for example_group in examples.values():
    example_group[['definition_overlap', 'examples_overlap', 'overlap']] = example_group.apply(calculate_overlap, axis=1, result_type='expand')

In [21]:
examples[4]

,definition,examples,generated,evaluated_generated,num_generated,unique_generated,definition_overlap,examples_overlap,overlap
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",['Ohne seine suggestiv - erhabenen Orchesterko...,"[""Es ist schwer, die Trennung zwischen dem har...","[Es ist schwer, die Trennung zwischen dem harm...",2,2,0.000,0.0,0.000
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...","['Sie fand es albern , ich hielt es für Kunst ...","[""Der Computerprogrammierer hatte eine Woche l...",[Der Computerprogrammierer hatte eine Woche la...,2,2,0.000,0.0,0.000
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...","['Wie die Ratskanzlei mitteilt , seien Patient...","[""Es ist erforderlich"", ""Es ist hilfreich"", ""E...","[Es ist erforderlich, Es ist hilfreich, Es ist...",7,7,0.000,0.0,0.000
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...","['"" "" In dieser Jahreszeit ist die Aedes - Müc...","[""Der Hund bellt am meisten und jagt am wenigs...",[Der Hund bellt am meisten und jagt am wenigst...,4,4,0.000,0.0,0.000
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...",['( darf ja im Grunde nichts schlechtes schrei...,"[""Weil ich schlecht bin"", ""Dafür habe ich mich...","[Weil ich schlecht bin, Dafür habe ich mich et...",4,4,0.000,0.0,0.000
...,...,...,...,...,...,...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...","['Ihre Partei , die Union , wird abermals zur ...","[""Wenn sie mit Abstand hinsichtlich ihrer Leis...",[Wenn sie mit Abstand hinsichtlich ihrer Leist...,4,4,0.125,0.0,0.125
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...","['Pfui ist das neue Hui , stern', 'Kate ist di...","[""Grau ist das neue Blond"", ""Weiß ist das neue...","[Grau ist das neue Blond, Weiß ist das neue Sc...",5,5,0.000,0.0,0.000
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...",['Berlin/Offenbach ( dpa ) - Sommer im April :...,"[""Der Gebrauch von ""pur"" in ""Tinte pur"" signal...","[""Der Gebrauch von ""pur"" in ""Tinte pur"" signal...",1,1,0.000,0.0,0.000
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...",['Der Sohn sei ständig » inmitten eines Ozeans...,"[""Die Kühe stehen auf dem Hof über die Hüte de...",[Die Kühe stehen auf dem Hof über die Hüte der...,4,4,0.000,0.0,0.000


In [20]:
examples[4]["overlap"].mean()

0.021774893841015344

In [26]:
for i, example_group in examples.items():
    example_group.to_csv(f"../../out/llama/{i}_shot_data.tsv", sep="\t", decimal=",")
    example_group.to_excel(f"../../out/llama/{i}_shot_data.xlsx")